In [3]:
import pandas as pd

csv_path = "/Users/georgepaul/Desktop/portfolio/SQL/Data_Ready_Model.csv"
df = pd.read_csv(csv_path)

print(f"Shape: {df.shape}")
print(df.head())

Shape: (1329, 15)
   EUI_kWh_per_sqmt         SQMT  NFLOOR  FLCEILHT  MONUSE  OCCUPYP  WKHRS  \
0        446.557221   19509.6300     994        12      12    100.0     70   
1        194.982475    2601.2840       5         8      12    100.0     45   
2        226.306350    2833.5415       3        10      12     43.0     50   
3        281.839592   17094.1520       3        12      12     92.0     38   
4        359.900019  139354.5000     995        12      12    100.0    168   

   NWKER  HEATP  COOLP  DAYLTP  HDD65  CDD65  YRCONC  PUBCLIM  
0    350  100.0  100.0     0.0   4463   1759       2        3  
1     12   80.0  100.0    15.0   2424    189       6        4  
2     25  100.0  100.0     0.0   4615   1184       7        3  
3    400  100.0  100.0    15.0   3943   1680       2        4  
4   2500   98.0   98.0     0.0   5545   1262       2        7  


In [4]:
import sqlite3
import pandas as pd
import os

# Paths
csv_path = "/Users/georgepaul/Desktop/portfolio/SQL/Data_Ready_Model.csv"
db_path = "/Users/georgepaul/Desktop/portfolio/SQL/model_data.db"

# Load and check
df = pd.read_csv(csv_path)
print(f"✅ CSV loaded: {df.shape}")

# Remove old DB (if broken)
if os.path.exists(db_path):
    os.remove(db_path)
    print("🗑️ Old DB removed")

# Write to SQLite
conn = sqlite3.connect(db_path)
df.to_sql("buildings", conn, if_exists="replace", index=False)
conn.close()

print("✅ New DB created with table: buildings")

✅ CSV loaded: (1329, 15)
🗑️ Old DB removed
✅ New DB created with table: buildings


In [5]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables found:", cursor.fetchall())
conn.close()

Tables found: [('buildings',)]
